In [1]:
#!pip3 install missing packages

import numpy as np
import re
import requests
import pandas as pd
import seaborn as sns
from functools import reduce

from igel import Igel

import matplotlib.pyplot as plt
from matplotlib import rcParams

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Helper Functions

In [2]:
def clean_dataset(df):
    #assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    #df.dropna(inplace=True)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df.replace(np.nan,0)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    df = df.replace(r'\D+', '', regex=True)
    return df
    #return df[indices_to_keep].astype(np.float64)
    

def clean_dataset_int(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df.replace(np.nan,0)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    df = df.replace(r'\D+', '', regex=True)
    return df[indices_to_keep].astype(np.float64)

## Mappings

Defining various mappings we'll use when manipulating the data.

In [3]:
priority_scorer = {
    '10' : 'Critical',
    '9' : 'Critical',
    '8' : 'Critical',
    '7' : 'High',
    '6' : 'High',
    '5' : 'Medium',
    '4' : 'Medium',
    '3' : 'Low',
    '2' : 'Low',
    '1.0' : 'Low',
    '0.0' : 'Low',
}

priority_mapping = {
    "Critical" : 10,
    "High" : 7.5,
    "Medium" : 5,
    "Low" : 2.5,
    "Unknown" : 0,
}

# What we consider to be highly important categories of information
highCategoriser = {
    0.0 : 'Other-Advice',
    1.0 : 'Other-Advice',
    2.0 : 'Report-CleanUp',
    3.0 : 'ContextualInformation',
    4.0 : 'Other-ContextualInformation',
    5.0 : 'CallToAction-Donations',
    6.0 : 'Report-EmergingThreats',
    7.0 : 'Report-Factoid',
    8.0 : 'Report-FirstPartyObservation',
    9.0 : 'Request-GoodsServices',
    10.0 : 'Report-Hashtags',
    11.0 : 'Request-InformationWanted',
    12.0 : 'Other-Irrelevant',
    13.0 : 'Report-Location',
    14.0 : 'CallToAction-MovePeople',
    15.0 : 'Report-MultimediaShare',
    16.0 : 'Report-NewSubEvent',
    17.0 : 'Report-News',
    18.0 : 'Report-Official',
    19.0 : 'Report-OriginalEvent',
    20.0 : 'Request-SearchAndRescue',
    21.0 : 'Other-Sentiment',
    22.0 : 'Report-ServiceAvailable',
    23.0 : 'Report-ThirdPartyObservation',
    24.0 : 'CallToAction-Volunteer',
    25.0 : 'Report-Weather',
    26.0 : 'hmm',
}

event_int_map =	{
  "guatemalaEarthquake2012": 7,
  "joplinTornado2011": 16,
  "athensEarthquake2020": 35,
  "baltimoreFlashFlood2020": 36,
  "brooklynBlockPartyShooting2020": 37,
  "daytonOhioShooting2020": 38,
  "elPasoWalmartShooting2020": 39,
  "gilroygarlicShooting2020": 40,
  "hurricaneBarry2020": 41,
  "indonesiaEarthquake2020": 42,
  "keralaFloods2020": 43,
  "myanmarFloods2020": 44,
  "papuaNewguineaEarthquake2020": 45,
  "siberianWildfires2020": 46,
  "typhoonKrosa2020": 47,
  "typhoonLekima2020": 48,
  "whaleyBridgeCollapse2020": 49
}

mymap = {'Advice':1, 'CleanUp':2, 'ContextualInformation':3, 'Discussion':4, 'Donations':5, 
        'EmergingThreats':6, 'Factoid':7, 'FirstPartyObservation':8, 'GoodsServices':9, 'Hashtags':10, 
        'InformationWanted':11,'Irrelevant':12, 'Location':13, 'MovePeople':14, 
         'MultimediaShare':15, 'NewSubEvent':16, 'News':17,
        'Official':18, 'OriginalEvent':19, 'SearchAndRescue':20, 'Sentiment':21, 'ServiceAvailable':22, 
         'ThirdPartyObservation':23,'Volunteer':24, 'Weather':25}

## FeatureAPI

Load the feature vector in from Play

In [4]:
import requests


#url = 'http://localhost:9000/stored_tweets'
import json
def parse_json_stream(stream):
    decoder = json.JSONDecoder()
    while stream:
        obj, idx = decoder.raw_decode(stream)
        yield obj
        stream = stream[idx:].lstrip()

url = 'http://tweetminer-2336003gproject.ida.dcs.gla.ac.uk/stored_tweets'

data = requests.get(url).text  

# delete unused parameters to parse to JSON format
dataFormat = data.replace("\r\n  ","").replace("\r\n","").replace("}\n{", "},\n{").replace(",}", "}").replace("\n", "")

dataFormat = dataFormat.replace(' \\"', " ").replace('\\" ', " ")

# convert to valid structure
dataFormat='{"datas":['+dataFormat+']}'

# convert to JSON format
dataJson = json.loads(dataFormat)

dfNonFormatte= pd.DataFrame(dataJson)

dfNonFormatte

# convert our JSON to DATAFRAM
a = 0
df = []
for element in dfNonFormatte["datas"]:
  df1 = dfNonFormatte["datas"][a] 
  df2 = pd.json_normalize(df1['tweets'])
  df.append(df2)
  a = a + 1

    
# Concat result
df = pd.concat(df, axis=0, join='outer', ignore_index=True)
df

,tweet_id,tweet_text,offset,embeddings,sentiment,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.bc,tfIdf.tsunami,tfIdf.papua,tfIdf.powerful,tfIdf.quake,tfIdf.guinea,tfIdf.earthquake,tfIdf.passed,tfIdf.threat,tfIdf.pacificocean,tfIdf.papuanewguinea,tfIdf.included,tfIdf.newguinea,tfIdf.staysafe,tfIdf.solomonislands,tfIdf.news,tfIdf.kitv,tfIdf.sports,tfIdf.amp,tfIdf.hawaii,tfIdf.channel,tfIdf.weather,tfIdf.honolulu,tfIdf.morning,tfIdf.britain,tfIdf.imagine,tfIdf.islands,tfIdf.engulfing,tfIdf.prayers,tfIdf.sea,tfIdf.hits,tfIdf.ring,tfIdf.alert,tfIdf.magnitude,tfIdf.issued,tfIdf.isolated,tfIdf.reported,tfIdf.hope,tfIdf.preliminary,tfIdf.lifted,tfIdf.warning,tfIdf.app,tfIdf.nothings,tfIdf.strong,tfIdf.earthbound,tfIdf.cmes,tfIdf.strikes,tfIdf.sun,tfIdf.coast,tfIdf.west,tfIdf.summary,tfIdf.kokopo,tfIdf.sse,tfIdf.gisborne,tfIdf.ne,tfIdf.report,tfIdf.zealand,tfIdf.ene,tfIdf.rocked,tfIdf.waves,tfIdf.epicenter,tfIdf.hazardous,tfIdf.coastal,tfIdf.massive,tfIdf.earthquakepapua,tfIdf.ripple,tfIdf.huge,tfIdf.portmoresby,tfIdf.usgs,tfIdf.radius,tfIdf.waved,tfIdf.abound,tfIdf.negative,tfIdf.paupau,tfIdf.earthquakes,tfIdf.magnet,tfIdf.geological,tfIdf.solomon,tfIdf.hit,tfIdf.survey,tfIdf.region,tfIdf.tuesday,tfIdf.rattled,tfIdf.earthquakemag,tfIdf.local,tfIdf.california,tfIdf.tsunamiwarning,tfIdf.read,tfIdf.thenews,tfIdf.cdt,tfIdf.occurred,tfIdf.tue,tfIdf.event,tfIdf.pacific,tfIdf.triggered,tfIdf.breaking,tfIdf.abc,tfIdf.island,tfIdf.residents,tfIdf.rattles,tfIdf.passes,tfIdf.cbc,tfIdf.repent,tfIdf.chance,tfIdf.israel,tfIdf.awakening,tfIdf.happening,tfIdf.god,tfIdf.country,tfIdf.stuck,tfIdf.nearby,tfIdf.triggering,tfIdf.central,tfIdf.nations,tfIdf.killed,tfIdf.subsiding,tfIdf.struck,tfIdf.hours,tfIdf.utc,tfIdf.erthquakes,tfIdf.volcanoes,tfIdf.ago,tfIdf.info,tfIdf.south,tfIdf.center,tfIdf.raised,tfIdf.late,tfIdf.depth,tfIdf.regionsolomon,tfIdf.location,tfIdf.magnitudemb,tfIdf.measured,tfIdf.papuanuevaguinea,tfIdf.northeast,tfIdf.australia,tfIdf.canberra,tfIdf.kilometers,tfIdf.located,tfIdf.evening,tfIdf.destructive,tfIdf.possibility,tfIdf.damage,tfIdf.images,tfIdf.expected,tfIdf.update,tfIdf.coming,tfIdf.papau,tfIdf.pray,tfIdf.fox,tfIdf.papa,tfIdf.animals,tfIdf.minutes,tfIdf.latestcomments,tfIdf.violent,tfIdf.shared,tfIdf.download,tfIdf.android,tfIdf.express,tfIdf.click,tfIdf.indian,tfIdf.expert,tfIdf.wise,tfIdf.hurricanes,tfIdf.bad,tfIdf.parties,tfIdf.hurricane,tfIdf.shots,tfIdf.prompting,tfIdf.prelim,tfIdf.reuters,tfIdf.prompts,tfIdf.lord,tfIdf.safe,tfIdf.expanded,tfIdf.usa,tfIdf.deep,tfIdf.solomons,tfIdf.tidal,tfIdf.risk,tfIdf.wave,tfIdf.overseas,tfIdf.moment,tfIdf.oceania,tfIdf.cancelled,tfIdf.headlines,tfIdf.developing,tfIdf.youtube,tfIdf.birthday,tfIdf.st,tfIdf.watch,tfIdf.israels,tfIdf.tsunamis,tfIdf.advertiser,tfIdf.star,tfIdf.poses,tfIdf.measuring,tfIdf.miles,tfIdf.earthquakeph,tfIdf.shallow,tfIdf.chain,tfIdf.north,tfIdf.knocking,tfIdf.safety,tfIdf.fleeing,tfIdf.sending,tfIdf.power,tfIdf.wake,tfIdf.rabaul,tfIdf.coasts,tfIdf.centre,tfIdf.epicentre,tfIdf.triggers,tfIdf.mag,tfIdf.eqalerts,tfIdf.ireland,tfIdf.newearthquake,...,tfIdf.audition,tfIdf.kurup,tfIdf.mahe,tfIdf.floodsheavy,tfIdf.kanhangad,tfIdf.horanadu,tfIdf.breached,tfIdf.muvattupuzha,tfIdf.mavoor,tfIdf.schoolsafety,tfIdf.schoolsout,tfIdf.nedumbassery,tfIdf.chaired,tfIdf.rajani,tfIdf

# Normalise the sentiment and the embeddings

> Extract the sentiment and embeddings into multiple columns depending on need                                                 

In [5]:
def expand_sentiment(df):
    df['sentiment'] = df[df['sentiment'].apply(lambda x:pd.Series(x))]
    df_sentiment=pd.concat([df['sentiment'].str.split(',', expand=True)], axis=1, keys="s")
    # Join back onto the original dataframe
    df.join(df_sentiment)
    return df
    
def expand_embeddings(df):
    df['embeddings'] = df['embeddings'].str.extract('(\d+)', expand=False)
    df['embeddings'] = df['embeddings'].str[1:]
    df_embeddings = pd.concat([df['embeddings'].str.split(',', expand=True)], axis=1, keys="e")
    df_embeddings
    df.join(df_embeddings) # Join back onto the original dataframe
    #df_emb_sent=df_sentiment.join(df_embeddings)
    #df_emb_sent
    #df.join(df_emb_sent)
    return df

In [6]:
df

,tweet_id,tweet_text,offset,embeddings,sentiment,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.bc,tfIdf.tsunami,tfIdf.papua,tfIdf.powerful,tfIdf.quake,tfIdf.guinea,tfIdf.earthquake,tfIdf.passed,tfIdf.threat,tfIdf.pacificocean,tfIdf.papuanewguinea,tfIdf.included,tfIdf.newguinea,tfIdf.staysafe,tfIdf.solomonislands,tfIdf.news,tfIdf.kitv,tfIdf.sports,tfIdf.amp,tfIdf.hawaii,tfIdf.channel,tfIdf.weather,tfIdf.honolulu,tfIdf.morning,tfIdf.britain,tfIdf.imagine,tfIdf.islands,tfIdf.engulfing,tfIdf.prayers,tfIdf.sea,tfIdf.hits,tfIdf.ring,tfIdf.alert,tfIdf.magnitude,tfIdf.issued,tfIdf.isolated,tfIdf.reported,tfIdf.hope,tfIdf.preliminary,tfIdf.lifted,tfIdf.warning,tfIdf.app,tfIdf.nothings,tfIdf.strong,tfIdf.earthbound,tfIdf.cmes,tfIdf.strikes,tfIdf.sun,tfIdf.coast,tfIdf.west,tfIdf.summary,tfIdf.kokopo,tfIdf.sse,tfIdf.gisborne,tfIdf.ne,tfIdf.report,tfIdf.zealand,tfIdf.ene,tfIdf.rocked,tfIdf.waves,tfIdf.epicenter,tfIdf.hazardous,tfIdf.coastal,tfIdf.massive,tfIdf.earthquakepapua,tfIdf.ripple,tfIdf.huge,tfIdf.portmoresby,tfIdf.usgs,tfIdf.radius,tfIdf.waved,tfIdf.abound,tfIdf.negative,tfIdf.paupau,tfIdf.earthquakes,tfIdf.magnet,tfIdf.geological,tfIdf.solomon,tfIdf.hit,tfIdf.survey,tfIdf.region,tfIdf.tuesday,tfIdf.rattled,tfIdf.earthquakemag,tfIdf.local,tfIdf.california,tfIdf.tsunamiwarning,tfIdf.read,tfIdf.thenews,tfIdf.cdt,tfIdf.occurred,tfIdf.tue,tfIdf.event,tfIdf.pacific,tfIdf.triggered,tfIdf.breaking,tfIdf.abc,tfIdf.island,tfIdf.residents,tfIdf.rattles,tfIdf.passes,tfIdf.cbc,tfIdf.repent,tfIdf.chance,tfIdf.israel,tfIdf.awakening,tfIdf.happening,tfIdf.god,tfIdf.country,tfIdf.stuck,tfIdf.nearby,tfIdf.triggering,tfIdf.central,tfIdf.nations,tfIdf.killed,tfIdf.subsiding,tfIdf.struck,tfIdf.hours,tfIdf.utc,tfIdf.erthquakes,tfIdf.volcanoes,tfIdf.ago,tfIdf.info,tfIdf.south,tfIdf.center,tfIdf.raised,tfIdf.late,tfIdf.depth,tfIdf.regionsolomon,tfIdf.location,tfIdf.magnitudemb,tfIdf.measured,tfIdf.papuanuevaguinea,tfIdf.northeast,tfIdf.australia,tfIdf.canberra,tfIdf.kilometers,tfIdf.located,tfIdf.evening,tfIdf.destructive,tfIdf.possibility,tfIdf.damage,tfIdf.images,tfIdf.expected,tfIdf.update,tfIdf.coming,tfIdf.papau,tfIdf.pray,tfIdf.fox,tfIdf.papa,tfIdf.animals,tfIdf.minutes,tfIdf.latestcomments,tfIdf.violent,tfIdf.shared,tfIdf.download,tfIdf.android,tfIdf.express,tfIdf.click,tfIdf.indian,tfIdf.expert,tfIdf.wise,tfIdf.hurricanes,tfIdf.bad,tfIdf.parties,tfIdf.hurricane,tfIdf.shots,tfIdf.prompting,tfIdf.prelim,tfIdf.reuters,tfIdf.prompts,tfIdf.lord,tfIdf.safe,tfIdf.expanded,tfIdf.usa,tfIdf.deep,tfIdf.solomons,tfIdf.tidal,tfIdf.risk,tfIdf.wave,tfIdf.overseas,tfIdf.moment,tfIdf.oceania,tfIdf.cancelled,tfIdf.headlines,tfIdf.developing,tfIdf.youtube,tfIdf.birthday,tfIdf.st,tfIdf.watch,tfIdf.israels,tfIdf.tsunamis,tfIdf.advertiser,tfIdf.star,tfIdf.poses,tfIdf.measuring,tfIdf.miles,tfIdf.earthquakeph,tfIdf.shallow,tfIdf.chain,tfIdf.north,tfIdf.knocking,tfIdf.safety,tfIdf.fleeing,tfIdf.sending,tfIdf.power,tfIdf.wake,tfIdf.rabaul,tfIdf.coasts,tfIdf.centre,tfIdf.epicentre,tfIdf.triggers,tfIdf.mag,tfIdf.eqalerts,tfIdf.ireland,tfIdf.newearthquake,...,tfIdf.audition,tfIdf.kurup,tfIdf.mahe,tfIdf.floodsheavy,tfIdf.kanhangad,tfIdf.horanadu,tfIdf.breached,tfIdf.muvattupuzha,tfIdf.mavoor,tfIdf.schoolsafety,tfIdf.schoolsout,tfIdf.nedumbassery,tfIdf.chaired,tfIdf.rajani,tfIdf

In [7]:
# Group by tweet
feature_vector_input = df.groupby(['tweet_id']).agg('first')
feature_vector_input

,tweet_text,offset,embeddings,sentiment,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.bc,tfIdf.tsunami,tfIdf.papua,tfIdf.powerful,tfIdf.quake,tfIdf.guinea,tfIdf.earthquake,tfIdf.passed,tfIdf.threat,tfIdf.pacificocean,tfIdf.papuanewguinea,tfIdf.included,tfIdf.newguinea,tfIdf.staysafe,tfIdf.solomonislands,tfIdf.news,tfIdf.kitv,tfIdf.sports,tfIdf.amp,tfIdf.hawaii,tfIdf.channel,tfIdf.weather,tfIdf.honolulu,tfIdf.morning,tfIdf.britain,tfIdf.imagine,tfIdf.islands,tfIdf.engulfing,tfIdf.prayers,tfIdf.sea,tfIdf.hits,tfIdf.ring,tfIdf.alert,tfIdf.magnitude,tfIdf.issued,tfIdf.isolated,tfIdf.reported,tfIdf.hope,tfIdf.preliminary,tfIdf.lifted,tfIdf.warning,tfIdf.app,tfIdf.nothings,tfIdf.strong,tfIdf.earthbound,tfIdf.cmes,tfIdf.strikes,tfIdf.sun,tfIdf.coast,tfIdf.west,tfIdf.summary,tfIdf.kokopo,tfIdf.sse,tfIdf.gisborne,tfIdf.ne,tfIdf.report,tfIdf.zealand,tfIdf.ene,tfIdf.rocked,tfIdf.waves,tfIdf.epicenter,tfIdf.hazardous,tfIdf.coastal,tfIdf.massive,tfIdf.earthquakepapua,tfIdf.ripple,tfIdf.huge,tfIdf.portmoresby,tfIdf.usgs,tfIdf.radius,tfIdf.waved,tfIdf.abound,tfIdf.negative,tfIdf.paupau,tfIdf.earthquakes,tfIdf.magnet,tfIdf.geological,tfIdf.solomon,tfIdf.hit,tfIdf.survey,tfIdf.region,tfIdf.tuesday,tfIdf.rattled,tfIdf.earthquakemag,tfIdf.local,tfIdf.california,tfIdf.tsunamiwarning,tfIdf.read,tfIdf.thenews,tfIdf.cdt,tfIdf.occurred,tfIdf.tue,tfIdf.event,tfIdf.pacific,tfIdf.triggered,tfIdf.breaking,tfIdf.abc,tfIdf.island,tfIdf.residents,tfIdf.rattles,tfIdf.passes,tfIdf.cbc,tfIdf.repent,tfIdf.chance,tfIdf.israel,tfIdf.awakening,tfIdf.happening,tfIdf.god,tfIdf.country,tfIdf.stuck,tfIdf.nearby,tfIdf.triggering,tfIdf.central,tfIdf.nations,tfIdf.killed,tfIdf.subsiding,tfIdf.struck,tfIdf.hours,tfIdf.utc,tfIdf.erthquakes,tfIdf.volcanoes,tfIdf.ago,tfIdf.info,tfIdf.south,tfIdf.center,tfIdf.raised,tfIdf.late,tfIdf.depth,tfIdf.regionsolomon,tfIdf.location,tfIdf.magnitudemb,tfIdf.measured,tfIdf.papuanuevaguinea,tfIdf.northeast,tfIdf.australia,tfIdf.canberra,tfIdf.kilometers,tfIdf.located,tfIdf.evening,tfIdf.destructive,tfIdf.possibility,tfIdf.damage,tfIdf.images,tfIdf.expected,tfIdf.update,tfIdf.coming,tfIdf.papau,tfIdf.pray,tfIdf.fox,tfIdf.papa,tfIdf.animals,tfIdf.minutes,tfIdf.latestcomments,tfIdf.violent,tfIdf.shared,tfIdf.download,tfIdf.android,tfIdf.express,tfIdf.click,tfIdf.indian,tfIdf.expert,tfIdf.wise,tfIdf.hurricanes,tfIdf.bad,tfIdf.parties,tfIdf.hurricane,tfIdf.shots,tfIdf.prompting,tfIdf.prelim,tfIdf.reuters,tfIdf.prompts,tfIdf.lord,tfIdf.safe,tfIdf.expanded,tfIdf.usa,tfIdf.deep,tfIdf.solomons,tfIdf.tidal,tfIdf.risk,tfIdf.wave,tfIdf.overseas,tfIdf.moment,tfIdf.oceania,tfIdf.cancelled,tfIdf.headlines,tfIdf.developing,tfIdf.youtube,tfIdf.birthday,tfIdf.st,tfIdf.watch,tfIdf.israels,tfIdf.tsunamis,tfIdf.advertiser,tfIdf.star,tfIdf.poses,tfIdf.measuring,tfIdf.miles,tfIdf.earthquakeph,tfIdf.shallow,tfIdf.chain,tfIdf.north,tfIdf.knocking,tfIdf.safety,tfIdf.fleeing,tfIdf.sending,tfIdf.power,tfIdf.wake,tfIdf.rabaul,tfIdf.coasts,tfIdf.centre,tfIdf.epicentre,tfIdf.triggers,tfIdf.mag,tfIdf.eqalerts,tfIdf.ireland,tfIdf.newearthquake,tfIdf.namatanai,...,tfIdf.audition,tfIdf.kurup,tfIdf.mahe,tfIdf.floodsheavy,tfIdf.kanhangad,tfIdf.horanadu,tfIdf.breached,tfIdf.muvattupuzha,tfIdf.mavoor,tfIdf.schoolsafety,tfIdf.schoolsout,tfIdf.nedumbassery,tfIdf.chaired,tfIdf.rajan

## Load the labelled data

These are generated in 0_Labels.ipynb

In [8]:
# dtypes needs to be specified or long ints will change
labels_df = pd.read_json("../../../data/input/raw/data/2020/2020-A/labels/TRECIS-2018-2020A-labels.json", dtype={} )

# replace the event with a numeric value
labels_df = labels_df.replace({'eventID': event_int_map})

# Count the number of labels
labels_df['num'] = labels_df['postCategories'].str.len()

# Map the priority to a numeric value
labels_df = labels_df.replace({"postPriority": priority_mapping})

# Split categories and map to numeric values
category_list = pd.DataFrame(labels_df["postCategories"].to_list(), columns=['cat1', 'cat2', 'cat3',
                                                                   'cat4', 'cat5', 'cat6',
                                                                   'cat7', 'cat8', 'cat9', 'cat10'])


# Map the categories to numeric values
category_list = category_list.applymap(lambda s: mymap.get(s) if s in mymap else s)


# Join back onto our original list
labels = labels_df.join(category_list)

# Drop the string categories
labels.drop(['postCategories'], axis = 1, inplace = True)

# Tidy
labels = labels.drop(['eventName', 'eventDescription', 'eventType'], axis=1)

# Fill the NaN slots with 0
labels = labels.fillna("0")

# Export
labels.to_csv("../labels.csv", index=False)
labels = clean_dataset_int(labels)
#labels = clean_dataset_new(labels)

labels
df = pd.merge(labels, feature_vector_input, left_on = 'postID', right_on = 'tweet_id', how = 'inner')
df

INFO - NumExpr defaulting to 8 threads.


,eventID,postID,postPriority,num,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,tweet_text,offset,embeddings,sentiment,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.bc,tfIdf.tsunami,tfIdf.papua,tfIdf.powerful,tfIdf.quake,tfIdf.guinea,tfIdf.earthquake,tfIdf.passed,tfIdf.threat,tfIdf.pacificocean,tfIdf.papuanewguinea,tfIdf.included,tfIdf.newguinea,tfIdf.staysafe,tfIdf.solomonislands,tfIdf.news,tfIdf.kitv,tfIdf.sports,tfIdf.amp,tfIdf.hawaii,tfIdf.channel,tfIdf.weather,tfIdf.honolulu,tfIdf.morning,tfIdf.britain,tfIdf.imagine,tfIdf.islands,tfIdf.engulfing,tfIdf.prayers,tfIdf.sea,tfIdf.hits,tfIdf.ring,tfIdf.alert,tfIdf.magnitude,tfIdf.issued,tfIdf.isolated,tfIdf.reported,tfIdf.hope,tfIdf.preliminary,tfIdf.lifted,tfIdf.warning,tfIdf.app,tfIdf.nothings,tfIdf.strong,tfIdf.earthbound,tfIdf.cmes,tfIdf.strikes,tfIdf.sun,tfIdf.coast,tfIdf.west,tfIdf.summary,tfIdf.kokopo,tfIdf.sse,tfIdf.gisborne,tfIdf.ne,tfIdf.report,tfIdf.zealand,tfIdf.ene,tfIdf.rocked,tfIdf.waves,tfIdf.epicenter,tfIdf.hazardous,tfIdf.coastal,tfIdf.massive,tfIdf.earthquakepapua,tfIdf.ripple,tfIdf.huge,tfIdf.portmoresby,tfIdf.usgs,tfIdf.radius,tfIdf.waved,tfIdf.abound,tfIdf.negative,tfIdf.paupau,tfIdf.earthquakes,tfIdf.magnet,tfIdf.geological,tfIdf.solomon,tfIdf.hit,tfIdf.survey,tfIdf.region,tfIdf.tuesday,tfIdf.rattled,tfIdf.earthquakemag,tfIdf.local,tfIdf.california,tfIdf.tsunamiwarning,tfIdf.read,tfIdf.thenews,tfIdf.cdt,tfIdf.occurred,tfIdf.tue,tfIdf.event,tfIdf.pacific,tfIdf.triggered,tfIdf.breaking,tfIdf.abc,tfIdf.island,tfIdf.residents,tfIdf.rattles,tfIdf.passes,tfIdf.cbc,tfIdf.repent,tfIdf.chance,tfIdf.israel,tfIdf.awakening,tfIdf.happening,tfIdf.god,tfIdf.country,tfIdf.stuck,tfIdf.nearby,tfIdf.triggering,tfIdf.central,tfIdf.nations,tfIdf.killed,tfIdf.subsiding,tfIdf.struck,tfIdf.hours,tfIdf.utc,tfIdf.erthquakes,tfIdf.volcanoes,tfIdf.ago,tfIdf.info,tfIdf.south,tfIdf.center,tfIdf.raised,tfIdf.late,tfIdf.depth,tfIdf.regionsolomon,tfIdf.location,tfIdf.magnitudemb,tfIdf.measured,tfIdf.papuanuevaguinea,tfIdf.northeast,tfIdf.australia,tfIdf.canberra,tfIdf.kilometers,tfIdf.located,tfIdf.evening,tfIdf.destructive,tfIdf.possibility,tfIdf.damage,tfIdf.images,tfIdf.expected,tfIdf.update,tfIdf.coming,tfIdf.papau,tfIdf.pray,tfIdf.fox,tfIdf.papa,tfIdf.animals,tfIdf.minutes,tfIdf.latestcomments,tfIdf.violent,tfIdf.shared,tfIdf.download,tfIdf.android,tfIdf.express,tfIdf.click,tfIdf.indian,tfIdf.expert,tfIdf.wise,tfIdf.hurricanes,tfIdf.bad,tfIdf.parties,tfIdf.hurricane,tfIdf.shots,tfIdf.prompting,tfIdf.prelim,tfIdf.reuters,tfIdf.prompts,tfIdf.lord,tfIdf.safe,tfIdf.expanded,tfIdf.usa,tfIdf.deep,tfIdf.solomons,tfIdf.tidal,tfIdf.risk,tfIdf.wave,tfIdf.overseas,tfIdf.moment,tfIdf.oceania,tfIdf.cancelled,tfIdf.headlines,tfIdf.developing,tfIdf.youtube,tfIdf.birthday,tfIdf.st,tfIdf.watch,tfIdf.israels,tfIdf.tsunamis,tfIdf.advertiser,tfIdf.star,tfIdf.poses,tfIdf.measuring,tfIdf.miles,tfIdf.earthquakeph,tfIdf.shallow,tfIdf.chain,tfIdf.north,tfIdf.knocking,tfIdf.safety,...,tfIdf.audition,tfIdf.kurup,tfIdf.mahe,tfIdf.floodsheavy,tfIdf.kanhangad,tfIdf.horanadu,tfIdf.breached,tfIdf.muvattupuzha,tfIdf.mavoor,tfIdf.schoolsafety,tfIdf.schoolsout,tfIdf.nedumbassery,tfIdf.chaired,tfIdf.rajani,tfIdf.incessantly,tfIdf.extremeweatherevents,tfIdf.southwestmonsoon,tfIdf.upward,tfIdf.preyed,tfIdf.nallathanni

In [9]:
df = expand_sentiment(df)

KeyboardInterrupt: 

In [ ]:
df = expand_embeddings(df)

In [ ]:
# Drop the string categories we no longer need
df.drop(['tweet_text', 'embeddings', 'sentiment'], axis = 1, inplace = True)

In [ ]:
feature_vector_input.drop(['tweet_text', 'embeddings', 'sentiment'], axis = 1, inplace = True)
feature_vector_input

# Train.csv


We merge the feature vector with the annotated data, this is used to train the classifier.

In [ ]:
# Merges the input feature vector with the labels
train = pd.merge(labels, feature_vector_input, left_on = 'postID', right_on = 'tweet_id', how = 'inner')

# reset the index
train = train.reset_index()

# fill NaN and replace Infinity
train.fillna(0, inplace=True)
train.replace([np.inf, -np.inf], np.nan, inplace=True)
train = train.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

# export to csv
train.to_csv("../train.csv", index=False)

train

# Test.csv


Drops the categories, number of categories and priority so we can make our prediction

In [ ]:
# Take a copy of the train dataframe
test = train

# Drop cat*
test.drop(list(test.filter(regex = 'cat\d+$')), axis = 1, inplace = True)

# Drop priority / num (of labels)
test.drop(['postPriority', 'num'], axis = 1, inplace = True)

# export
test.to_csv("../test.csv", index=False)

test

# Igel

This notebook implements `Igel` and supports all sklearn's machine learning functionality.

Caution must be taken to avoid overfitting. See `docs/ml.md` for more information

Igel's supported models:

        +--------------------+----------------------------+-------------------------+
        |      regression    |        classification      |        clustering       |
        +--------------------+----------------------------+-------------------------+
        |   LinearRegression |         LogisticRegression |                  KMeans |
        |              Lasso |                      Ridge |     AffinityPropagation |
        |          LassoLars |               DecisionTree |                   Birch |
        | BayesianRegression |                  ExtraTree | AgglomerativeClustering |
        |    HuberRegression |               RandomForest |    FeatureAgglomeration |
        |              Ridge |                 ExtraTrees |                  DBSCAN |
        |  PoissonRegression |                        SVM |         MiniBatchKMeans |
        |      ARDRegression |                  LinearSVM |    SpectralBiclustering |
        |  TweedieRegression |                      NuSVM |    SpectralCoclustering |
        | TheilSenRegression |            NearestNeighbor |      SpectralClustering |
        |    GammaRegression |              NeuralNetwork |               MeanShift |
        |   RANSACRegression | PassiveAgressiveClassifier |                  OPTICS |
        |       DecisionTree |                 Perceptron |                    ---- |
        |          ExtraTree |               BernoulliRBM |                    ---- |
        |       RandomForest |           BoltzmannMachine |                    ---- |
        |         ExtraTrees |       CalibratedClassifier |                    ---- |
        |                SVM |                   Adaboost |                    ---- |
        |          LinearSVM |                    Bagging |                    ---- |
        |              NuSVM |           GradientBoosting |                    ---- |
        |    NearestNeighbor |        BernoulliNaiveBayes |                    ---- |
        |      NeuralNetwork |      CategoricalNaiveBayes |                    ---- |
        |         ElasticNet |       ComplementNaiveBayes |                    ---- |
        |       BernoulliRBM |         GaussianNaiveBayes |                    ---- |
        |   BoltzmannMachine |      MultinomialNaiveBayes |                    ---- |
        |           Adaboost |                       ---- |                    ---- |
        |            Bagging |                       ---- |                    ---- |
        |   GradientBoosting |                       ---- |                    ---- |
        +--------------------+----------------------------+-------------------------+

In [ ]:
# Train the model

params = {
        'cmd': 'fit',    
        'data_path': "../train.csv",
        'yaml_path': 'yaml/multi.yaml'  # DecisionTree
}

Igel(**params)

In [ ]:
# Use model to predict on missing values

params = {
        'cmd': 'predict',    
        'data_path': "../test.csv",
        'yaml_path': 'yaml/hyper.yaml'
}
Igel(**params)

# Predictions

1. View the raw predictions
2. Map the labels to their High Level Information Types
3. Merge the predictions back into the training set


In [ ]:
def is_neg_predictions(predictions):
    predictions = predictions.sort_values(by=['postPriority'])
    predictions = predictions[(predictions > 0).all(1)]
    predictions.round()

predictions = pd.read_csv("model_results/predictions.csv")
predictions

#is_neg_predictions(predictions)

### Merge the new predictions back onto dataframe with the missing columns

In [ ]:
# Map the labels to their High Level Information Types
cat_list = predictions.filter(regex='cat', axis=1).round().applymap(lambda x: highCategoriser[x])

#
predictions = cat_list.combine_first(predictions)

# Merge the predictions back into the training set
df = test.merge(predictions, left_index=True, right_index=True)


# Append the predicted categories to a list in a new column
df['predicted_categories'] = df[['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10']].values.tolist()


# Get the number of categories into something we can use to index
df['num'] = df['num'].astype(float).astype(int)

# Remove categories beyond what the tweet is predicted to have
df['categories'] = df.apply(lambda x: x['predicted_categories'][0:x['num']], axis=1)

df

## Export

Export in the TRECIS format

In [ ]:
# write to .run file
with open("marks2.run" , "w") as out_file:
    for row in df.drop_duplicates(subset="postID").itertuples():
        #print("row:", row)
        content = [
            "TRECIS-CTIT-H-Test-0" + str(int(row.eventID)),
            "Q0",          
            np.int64(row.postID),   
            getattr(row, 'Index'),  #ToDo: Fix?
            #row.priority,
            str(priority_scorer[str(round(row.postPriority))[:3]]),  #ToDo: Fix
            row.categories,
            "marksrun2"
        ]
        out_file.write("\t".join([str(x) for x in content]) + "\n")

In [ ]:
#

In [ ]:
#df['tweets'] = df['tweets'].str[2:-1]
df


In [ ]:
# JSONDecodeError: Expecting ',' delimiter: 
#json_response = json.loads('[' + q.text + '],')

#print(data)

In [ ]:
#file_list = ["../output/run2.json"] #"../output/run.json", "../output/run0.json", 
#dfs = [] # an empty list to store the data frames

#for file in file_list:
    with open(file) as f:
        json_data = pd.json_normalize(json.loads(f.read()))
    dfs.append(json_data)
df = pd.concat(dfs, sort=False) # or sort=True depending on your needs
df




In [ ]:
#df = pd.read_json("../output/run.json", lines=True)
from pandas.io.json import json_normalize
df = pd.concat([pd.DataFrame(json_normalize(x)) for x in df['tweets']],ignore_index=True)
df